## Getis ord con Pysal

### 1. Preparar los datos
Asumiendo que tienes una tabla df con las siguientes columnas:
- postal_code: Código postal.
- positivity_rate: Tasa de positividad ajustada por la población de mascotas. Para conseguir este número, hay que sumar todos los casos positivos y strong positive de un mismo CP y luego dividir por la estimación de la población de mascotas de ese CP (esto es una posible normalización)
- geometry: Geometría de los códigos postales en formato de polígonos (esto se puede obtener con geopandas).


In [ ]:
# imports
import pandas as pd

import geopandas as gpd
from libpysal.weights import DistanceBand

from esda import G_Local
import numpy as np

In [30]:
df = pd.read_csv("data/new_data.csv", dtype={'Postal code': str})

# fitler to get two columns
df = df[['Value test', 'Postal code']]

# change postal codes with 4 digits to 5 addin a 0 in the left
df['Postal code'] = df['Postal code'].str.zfill(5)

print(df.head(1))

  Value test Postal code
0   Negativo       08520


In [32]:
# load postal codes shapefile to get polygon
gdf = gpd.read_file("data/codigos_postales/codigos_postales.shp")
gdf = gdf.rename(columns={'COD_POSTAL': 'Postal code'})
gdf = gdf[['Postal code', 'geometry']]
print(gdf.head(1))

  Postal code                                           geometry
0       01193  POLYGON ((-2.5306 42.81828, -2.53061 42.81814,...


In [34]:
# merge both dataframes to work with df
df_merged = pd.merge(df, gdf, on='Postal code', how='left')

print(df_merged.head())

  Value test Postal code                                           geometry
0   Negativo       08520  POLYGON ((2.34897 41.67408, 2.34883 41.67185, ...
1   Negativo       08520  POLYGON ((2.34897 41.67408, 2.34883 41.67185, ...
2   Negativo       08520  POLYGON ((2.34897 41.67408, 2.34883 41.67185, ...
3   Negativo       08520  POLYGON ((2.34897 41.67408, 2.34883 41.67185, ...
4   Negativo       08520  POLYGON ((2.34897 41.67408, 2.34883 41.67185, ...


### 2. Crear la Matriz de Ponderación Espacial
Para crear una matriz de ponderación espacial, puedes definir los vecinos por distancia usando un umbral específico (por ejemplo, 5 km). PySAL proporciona métodos para generar esta matriz de adyacencia.